In [49]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

legisladores_df = pd.read_csv('data/legisladores.csv')
legisladores_df.head()

,cuerpo,email,id_legislador,lema,nombre
0,NaN,eagazzi@parlamento.gub.uy,5146,NaN,"Agazzi, Ernesto"
1,NaN,NaN,2743,NaN,"Alcorta, Ricardo"
2,NaN,secretaria.ministro@mrree.gub.uy,8883,NaN,"Almagro Lemes, Luis Leonardo"
3,SENADO,valonso@parlamento.gub.uy,9891,PARTIDO NACIONAL,"Alonso, Verónica"
4,NaN,nolivera@parlamento.gub.uy,11535,NaN,"Álvarez López, María Dolores"


## Dimensiones

### Asistencia
Presiencia en el plenario y comisiones<br>
Promedio de la presencia en el plenario y las comisiones (asistencias / citaciones)

In [50]:
asistencia_plenario_df = pd.read_csv('data/asistencia_plenario.csv')
asistencia_comision_df = pd.read_csv('data/asistencia_comisiones.csv')
asistencia_df = pd.concat([asistencia_plenario_df[['id_legislador', 'asistencias', 'citaciones']], asistencia_comision_df[['id_legislador', 'asistencias', 'citaciones']]])
asistencia_df = asistencia_df.groupby('id_legislador').sum()

# Cálculo de z-score de asistencias
asistencia_df.loc[:,'asistencia_med'] = asistencia_df.apply(lambda x : x['asistencias'] * 1.0 / x['citaciones'], axis=1)
asistencia_media = asistencia_df['asistencia_med'].mean()
asistencia_std = asistencia_df['asistencia_med'].std()
asistencia_df.loc[:,'asistencia_zidx'] = asistencia_df.apply(lambda x : (x['asistencia_med'] * 1.0 - asistencia_media) / asistencia_std, axis=1)
asistencia_df.reset_index(inplace=True)
asistencia_df = asistencia_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
asistencia_df.head()

,id_legislador,asistencias,citaciones,asistencia_med,asistencia_zidx,nombre,cuerpo,lema
0,149,414,454,0.911894,0.228856,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,376,492,0.764228,-0.626013,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,558,665,0.839098,-0.192577,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,359,463,0.775378,-0.561462,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,364,446,0.816143,-0.325463,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


### Proyectos presentados
Incluye las siguientes actividades:
* Presentación o firma de proyectos de ley
* Proyectos de resolución
* Proyectos de minutas
* Proyectos de declaración de cuerpo
* Pedidos de informe
<br><br>El índice se basa en la suma de todos los proyectos presentados

In [51]:
proyectos_presentados_df = pd.read_csv('data/proyectos_presentados.csv')
proyectos_df = pd.read_csv('data/proyectos.csv')
proyectos_presentados_df = proyectos_presentados_df.merge(proyectos_df, on='id_proyecto', how='left')

proyectos_presentados_df = proyectos_presentados_df[proyectos_presentados_df['tipo'].isin(['PEDIDO DE INFORMES', 'PROYECTO DE DECLARACION', 'PROYECTO DE LEY', 'PROYECTO DE MINUTA DE COMUNICACION','PROYECTO DE RESOLUCION'])]
proyectos_presentados_df = proyectos_presentados_df[['id_legislador', 'id_proyecto']].drop_duplicates().groupby('id_legislador').size().reset_index(name='proyectos_total')

proyectos_presentados_media = proyectos_presentados_df['proyectos_total'].mean()
proyectos_presentados_std = proyectos_presentados_df['proyectos_total'].std()
proyectos_presentados_df.loc[:,'proyectos_presentados_zidx'] = proyectos_presentados_df.apply(lambda x : (x['proyectos_total'] * 1.0 - proyectos_presentados_media) / proyectos_presentados_std, axis=1)
proyectos_presentados_df = proyectos_presentados_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
proyectos_presentados_df.head()

,id_legislador,proyectos_total,proyectos_presentados_zidx,nombre,cuerpo,lema
0,149,57,2.203994,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,31,0.823760,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,31,0.823760,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,42,1.407705,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,16,0.027471,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


### Informante en comisión
Informantes de proyectos en las comisiones en las que participa<br>
El índice se basa en la suma de número de acciones

In [52]:
actuacion_parlamentaria_df = pd.read_csv('data/actuacion_parlamentaria.csv', index_col=0)
informante_comision_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INFORMA']
informante_comision_df = informante_comision_df.groupby('id_legislador').size().reset_index(name='informes_total')
informante_comision_media = informante_comision_df['informes_total'].mean()
informante_comision_std = informante_comision_df['informes_total'].std()
informante_comision_df.loc[:,'informe_comision_zidx'] = informante_comision_df.apply(lambda x : (x['informes_total'] * 1.0 - informante_comision_media) / informante_comision_std, axis=1)
informante_comision_df = informante_comision_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
informante_comision_df.head()

,id_legislador,informes_total,informe_comision_zidx,nombre,cuerpo,lema
0,149,10,0.136277,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,1,-0.571700,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,27,1.473569,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,9,0.057613,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,13,0.372270,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


### Debate
Interviene en debates sobre proyectos<br>
El índice se basa en la suma de las intervenciones en el plenario

In [53]:
interviene_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'INTERVIENE']
interviene_df = interviene_df.groupby('id_legislador').size().reset_index(name='interveciones_total')
interviene_media = interviene_df['interveciones_total'].mean()
interviene_std = interviene_df['interveciones_total'].std()
interviene_df.loc[:,'intervenciones_zidx'] = interviene_df.apply(lambda x : (x['interveciones_total'] * 1.0 - interviene_media) / interviene_std, axis=1)
interviene_df = interviene_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
interviene_df.head()


,id_legislador,interveciones_total,intervenciones_zidx,nombre,cuerpo,lema
0,149,296,3.342141,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,132,1.170970,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,325,3.726068,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,230,2.468377,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,117,0.972388,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


### Exposiciones
Escribe notas o realiza exposiciones sobre temas determinados.<br>
El índice se basa en la suma de las exposiciones realizadas

In [54]:
expone_df = actuacion_parlamentaria_df[actuacion_parlamentaria_df['tipo'] == 'EXPONE']
expone_df = expone_df.groupby('id_legislador').size().reset_index(name='exposiciones_total')
expone_media = expone_df['exposiciones_total'].mean()
expone_std = expone_df['exposiciones_total'].std()
expone_df.loc[:,'exposiciones_zidx'] = expone_df.apply(lambda x : (x['exposiciones_total'] * 1.0 - expone_media) / expone_std, axis=1)
expone_df = expone_df.merge(legisladores_df[['id_legislador', 'nombre', 'cuerpo', 'lema']], on='id_legislador')
expone_df.head()


,id_legislador,exposiciones_total,exposiciones_zidx,nombre,cuerpo,lema
0,149,25,0.650913,"Pasquet Iribarne, Ope",REPRESENTANTE,PARTIDO COLORADO
1,196,24,0.599328,"Viera Duarte, Tabaré",REPRESENTANTE,PARTIDO COLORADO
2,197,23,0.547743,"Heber, Luis Alberto",SENADO,PARTIDO NACIONAL
3,260,5,-0.380794,"Gandini, Jorge",REPRESENTANTE,PARTIDO NACIONAL
4,277,19,0.341401,"Carámbula Volpi, Marcos Gustavo",NaN,NaN


## Índice de legisladores

In [55]:
indice_senadores_df = asistencia_df[['nombre', 'cuerpo', 'lema', 'id_legislador', 'asistencia_zidx']]
indice_senadores_df = indice_senadores_df.merge(proyectos_presentados_df[['id_legislador', 'proyectos_presentados_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(informante_comision_df[['id_legislador', 'informe_comision_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(interviene_df[['id_legislador', 'intervenciones_zidx']], on='id_legislador')
indice_senadores_df = indice_senadores_df.merge(expone_df[['id_legislador', 'exposiciones_zidx']], on='id_legislador')
indice_senadores_df.loc[:, 'zidx'] = indice_senadores_df['asistencia_zidx'] +indice_senadores_df['asistencia_zidx'] + indice_senadores_df['proyectos_presentados_zidx'] +indice_senadores_df['informe_comision_zidx'] + indice_senadores_df['intervenciones_zidx'] + indice_senadores_df['exposiciones_zidx']
indice_senadores_df.sort_values('zidx', ascending=False).head()


,nombre,cuerpo,lema,id_legislador,asistencia_zidx,proyectos_presentados_zidx,informe_comision_zidx,intervenciones_zidx,exposiciones_zidx,zidx
14,"Bordaberry Herrán, Juan Pedro",SENADO,PARTIDO COLORADO,922,0.167671,2.469424,1.866890,7.618289,1.888962,14.178908
7,"Martínez Huelmo, Rubén",SENADO,PARTIDO FRENTE AMPLIO,408,0.489129,0.133643,7.137391,2.666960,0.599328,11.515581
25,"Abdala, Pablo",REPRESENTANTE,PARTIDO NACIONAL,3062,-0.000128,2.947197,0.136277,4.930803,1.940547,9.954568
80,"Arocena Argul, José Andrés",REPRESENTANTE,PARTIDO NACIONAL,9953,-1.133596,2.894111,-0.178379,0.839999,7.872863,9.161401
5,"Mieres, Pablo",SENADO,PARTIDO INDEPENDIENTE,327,0.018731,2.469424,0.529598,4.599832,0.805669,8.441985
